In [12]:
%load_ext autoreload
%autoreload 2

from entities.ambient import Ambient

ambient = Ambient(
     name="Ambient Test",
     ambient_images_path=[
          "./assets/img/ambient_1.png", 
          "./assets/img/ambient_2.png", 
          "./assets/img/ambient_3.png", 
          "./assets/img/ambient_4.png"], 
     audio_effect_path= None, #"./assets/audio/Street Sound Effect.mp3",
)

ambient.start()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
